## This notebook is an example of how to pipeline two models. 
A video stream from a local camera is processed by the face detection model. The face detection results are then processed by the mask detection model, one face bounding box at a time. Combined result is then displayed.

This example uses `mystreams` streaming toolkit.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

**Access to camera is required to run this sample.**

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in [env.ini](env.ini) file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

#### Specify camera index 

In [ ]:
camera_id = None         # camera index or URL; 0 to use default local camera, None to take from env.ini file

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = mytools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, mytools.get_cloud_zoo_url(), mytools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_det_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")

# adjust mask model properties
mask_det_model.output_top_k = 1 # report single result with top confidence

# adjust face model properties
face_det_model.overlay_show_probabilities = True
face_det_model.output_confidence_threshold = 0.7

In [ ]:
from mystreams import *

# Define mask detection gizmo (in mystreams terminology)
class MaskDetectionGizmo(AiGizmoBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cur_result = None
        
    def on_result(self, result):
        
        # here result.info contains StreamData object used for AI inference (because AiGizmoBase does it this way);
        # and result.info.meta contains metainfo dictionary placed by AiObjectDetectionCroppingGizmo, 
        # because in our pipeline it is connected as a source of this gizmo
        meta = result.info
        if "original_result" in meta: # new frame comes
            if self._cur_result is not None:
                # send previous frame
                self.send_result(StreamData(self._cur_result.image, self._cur_result))                
            self._cur_result = meta["original_result"]
        
        if "cropped_index" in meta:
            # apply mask presence/absence label
            if len(result.results) > 0:
                self._cur_result.results[meta["cropped_index"]]["label"] = result.results[0]["label"]
                self._cur_result.results[meta["cropped_index"]]["score"] = result.results[0]["score"]

In [ ]:
# create composition object
c = Composition()

# create gizmos adding them to composition
source = c.add(VideoSourceGizmo(camera_id))  # video source
face_detection = c.add(AiSimpleGizmo(face_det_model))  # face detection gizmo
face_crop = c.add(
    AiObjectDetectionCroppingGizmo(["face"])
)  # cropping gizmo which outputs cropped image for each detected face
mask_detection = c.add(MaskDetectionGizmo(mask_det_model))  # mask detection gizmo
face_display = c.add(
    VideoDisplayGizmo("Faces", show_ai_overlay=True, show_fps=True)
)  # display

# connect gizmos to create pipeline
source >> face_detection
face_crop.connect_to(source, 0)
face_crop.connect_to(face_detection, 1)
face_crop >> mask_detection >> face_display

# start execution of composition
c.start()
